In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb

In [27]:
train_targets = pd.read_parquet('data/A/parquet/train_targets.parquet', engine='pyarrow')
x_test_estimated = pd.read_parquet('data/A/parquet/X_test_estimated.parquet', engine='pyarrow')
x_train_estimated = pd.read_parquet('data/A/parquet/X_train_estimated.parquet', engine='pyarrow')
x_train_observed = pd.read_parquet('data/A/parquet/X_train_observed.parquet', engine='pyarrow')

# merge x_train_observed and x_train_estimated and remove the first column in x_train_estimated
x_train = pd.concat([x_train_observed, x_train_estimated.iloc[:, 1:]], axis=0)

# Set indicies to Datetime
x_train.set_index('date_forecast', inplace=True)
train_targets.set_index('time', inplace=True)

x_train.head()

,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
date_forecast,,,,,,,,,,,,,,,,,,,,,
2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,0.0,...,342.834015,-3.202,0.0,285.899994,100.0,39640.101562,3.7,-3.6,-0.8,-0.0
2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,0.0,...,346.294006,-3.650,0.0,286.100006,100.0,40123.898438,3.6,-3.6,-0.6,-0.0
2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,0.0,...,349.768005,-3.998,0.0,286.299988,100.0,40628.300781,3.6,-3.6,-0.4,-0.0
2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,0.0,...,353.251007,-4.247,0.0,286.600006,100.0,41153.601562,3.5,-3.5,-0.2,-0.0
2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,0.0,...,356.742004,-4.393,0.0,286.799988,100.0,41699.898438,3.5,-3.5,0.0,-0.0


In [28]:
# Create features of train_targets
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    
    return df

x_train = create_features(x_train)
train_targets = create_features(train_targets)

In [30]:
# Create features and targets
FEATURES = list(x_train.columns)
TARGET = ['pv_measurement']

X_train = x_train[FEATURES]
y_train = train_targets[TARGET]


X_train['pv_measurement'] = train_targets['pv_measurement']
X_train = X_train[~y_train['pv_measurement'].isna()]
y_train = y_train[~y_train['pv_measurement'].isna()]

/tmp/ipykernel_24659/2535990083.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  X_train = X_train[~y_train['pv_measurement'].isna()]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [ ]:
learning_rate = 0.2
max_depth = 9
min_child_weight = 10
n_estimators = 800

In [ ]:
reg = xgb.XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, min_child_weight=min_child_weight)
reg.fit(X_train, y_train)

XGBoostError: [16:04:00] /workspace/src/data/data.cc:501: Check failed: this->labels.Size() % this->num_row_ == 0 (34085 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/stinky/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x3581ea) [0x7fab71d581ea]
  [bt] (1) /home/stinky/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x389457) [0x7fab71d89457]
  [bt] (2) /home/stinky/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x38a4b1) [0x7fab71d8a4b1]
  [bt] (3) /home/stinky/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xb0) [0x7fab71b5e210]
  [bt] (4) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7fabd7920e2e]
  [bt] (5) /lib/x86_64-linux-gnu/libffi.so.8(+0x4493) [0x7fabd791d493]
  [bt] (6) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0xa3e9) [0x7fabd76363e9]
  [bt] (7) /usr/lib/python3.10/lib-dynload/_ctypes.cpython-310-x86_64-linux-gnu.so(+0x9a00) [0x7fa